In [17]:
import pymssql
import re
from collections import OrderedDict
from pandas import Series,DataFrame
from tqdm import tqdm_notebook
import pandas as pd
import copy

import urllib.request #이미지 다운로드
import numpy as np

import pickle

import tensorflow as tf


import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding


In [18]:
#피클 파일 불러오기 
with open('shoda4000.pickle','rb') as fr: 
    result3 = pickle.load(fr)

In [19]:
#인덱스 섞어주기
result3 = pd.concat([result3],ignore_index=True)
result3 = result3.reindex(np.random.permutation(result3.index)) 

In [20]:
#원핫벡터 정답 label 만들어주기
from keras.utils.np_utils import to_categorical
labels = to_categorical(result3['LABEL'], num_classes =8) 
print(labels[:10])

[[ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.]]


In [21]:
#토크나이징
vocabulary_size = 10000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(result3['pl_goodsnm'])

sequences= tokenizer.texts_to_sequences(result3['pl_goodsnm'])
data = pad_sequences(sequences, maxlen=10)
#데이터셋 분할
from sklearn.model_selection import train_test_split
data_train,data_test,labels_train,labels_test = train_test_split(data,labels, test_size = 0.25, random_state=42)

In [22]:
#임벨런스드 데이터 오버 샘플링 - 시간 오래걸림, 성능개선x
# from imblearn.over_sampling import SMOTE 
# sm = SMOTE(random_state=12)
# data_train_sm, labels_train_sm = sm.fit_sample(data_train,labels_train)

In [ ]:
#LSTM모델
model = Sequential()
model.add(Embedding(10000, 200, input_length=10))
model.add(LSTM(10, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(8,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [ ]:
#모델 학습
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping()
hist = model.fit(data_train,labels_train, epochs=10,batch_size = 100, validation_split = 0.2,callbacks=[early_stopping])
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')

Train on 13124 samples, validate on 3281 samples
Epoch 1/10
13124/13124 [==============================] - 4s 337us/step - loss: 1.6871 - acc: 0.5082 - val_loss: 1.1833 - val_acc: 0.7230
Epoch 2/10
13124/13124 [==============================] - 4s 269us/step - loss: 0.8659 - acc: 0.7966 - val_loss: 0.7191 - val_acc: 0.8226
Epoch 3/10
13124/13124 [==============================] - 4s 268us/step - loss: 0.5520 - acc: 0.8667 - val_loss: 0.6375 - val_acc: 0.8333
Epoch 4/10
13124/13124 [==============================] - 4s 269us/step - loss: 0.4201 - acc: 0.8958 - val_loss: 0.6174 - val_acc: 0.8379
Epoch 5/10
10300/13124 [======================>.......] - ETA: 0s - loss: 0.3465 - acc: 0.9139

In [ ]:
#성능 측정
accr = model.evaluate(data_test,labels_test)

In [ ]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
#학습 그래프
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()